In [7]:
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import torch

In [2]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols , channels)

z_dim = 100


In [3]:
def build_generator(z_dim):
    
    model = Sequential()
    
    model.add(Dense(256* 7 *7 , input_dim = z_dim))
    model.add(Reshape((7, 7 , 256)))
    
    model.add(Conv2DTranspose(128 , kernel_size = 3, strides = 2 , padding = 'same'))
    
    model.add(BatchNormalization())
    
    model.add(LeakyReLU(alpha = 0.01))
    
    model.add(Conv2DTranspose(64 , kernel_size = 3, strides =1 , padding = 'same'))
    
    model.add(BatchNormalization())
    
    model.add(LeakyReLU(alpha = 0.01))
    
    model.add(Conv2DTranspose(1, kernel_size = 3, strides =2, padding = 'same'))
    
    model.add(Activation('tanh'))
    
    return model

In [4]:
def build_discriminator(img_shape):
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size = 3, strides = 2, input_shape = img_shape, padding = 'same'))
    
    
    model.add(LeakyReLU(alpha = 0.01))
    
    model.add(Conv2D(64 , kernel_size = 3 , strides = 2, padding = 'same'))
    
    model.add(LeakyReLU(alpha = 0.01))
    
    model.add(Conv2D(128, kernel_size = 3, strides = 2 , padding = 'same'))
    
    model.add(LeakyReLU(alpha = 0.01))
    
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))
    
    return model

In [5]:
def build_gan(generator, discriminator):
    
    model = Sequential()
    
    model.add(generator)
    model.add(discriminator)
    
    return model

discriminator = build_discriminator(img_shape)
discriminator.compile(loss = 'binary_crossentropy' ,
                      optimizer = Adam(),
                      metrics = ['accuracy'])

generator = build_generator(z_dim)

discriminator.trainable = False

gan = build_gan(generator, discriminator)
gan.compile(loss = 'binary_crossentropy' , optimizer = Adam())

losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size , sample_interval):
    
    (x_train, _ ), (_,_ ) = mnist.load_data()
    
    x_train = x_train / 127.5 - 1.0
    x_train = np.expand_dims(x_train, axis = 3)
    
    real = np.ones((batch_size , 1))
    
    fake = np.zeros((batch_size , 1))
    
    for iteration in range(iterations):
        
        idx = np.random.randint( 0 , x_train.shape[0] , batch_size)
        imgs = x_train[idx]
        
        z = np.random.normal(0,1,(batch_size , 100))
        gen_imgs = generator.predict(z)
        
        d_loss_real = discriminator.train_on_batch(imgs,real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss , accuracy = 0.5 * np.add(d_loss_real , d_loss_fake)
        
        z = np.random.normal( 0 ,1 , (batch_size , 100))
        gen_imgs = generator.predict(z)
        
        g_loss = gan.train_on_batch(z, real)
        
        if ((iteration + 1) % sample_interval ) == 0:
            
            losses.append((d_loss,g_loss))
            accuracies.append(100.0 *accuracy)
            iteration_checkpoints.append(iteration +1)
            
            print("%d [D 손실 : %f , 정확도: %.2f%%] [ G 솔실: %f]" % (iteration +1 , d_loss , 100.0* accuracy , g_loss))
            
            sample_images(generator)

In [6]:
def sample_images(generator , image_grid_rows = 4 , image_grid_columns =4):
    
    z = np.random.normal( 0 , 1 , (image_grid_rows * image_grid_columns , z_dim))
    
    gen_imgs = generator.predict(z)
    
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig , axs = plt.subplots(image_grid_rows , image_grid_columns , figsize = (4,4), sharey = True , sharex = True)
    
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[ i , j].imshow(gen_imgs[cnt, : , : , 0] , cmap = 'gray')
            axs[ i , j].axis('off')
            cnt +=1
            
    plt.show()

In [8]:
iterations = 20000
batch_size = 64
sample_interval = 100


train(iterations , batch_size , sample_interval).to(device)

KeyboardInterrupt: 